Importing Libraries


In [ ]:
!pip install spark-nlp
!pip install pyspark
!pip install PyPDF2
from PyPDF2 import PdfReader as pdr
import sparknlp
import pandas as pd
# Start Spark Session
spark = sparknlp.start()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
reader = pdr('ExampleNotes.pdf')

print(len(reader.pages))

# creating a page object
page = reader.pages[0]
#page1 = reader.pages[1]
#page2 = reader.pages[2]

# extracting text from page
data = page.extract_text()
#data = data + '\nl' + page1.extract_text()
#data = data + '\nl' + page2.extract_text()
print(data)

1
Grave of the Fireﬂies Notes 
How does the ﬁlm portray the human cost of war, particularly through the lens of children? 
 
The human cost of war is portrayed brutally. It isn't overdramatized, it just happens. One 
scene, Japan looks ﬁne, but a few scenes later, the place is in ruins and mass graves are 
being erected. 
Through the lens of children, we don't just see the unnecessary death toll of war, we see 
the intense shock of losing family at a young age.  
The possibility of losing your family isn't a terribly common thought to a young child, so to 
experience it without warning or preparation is life-changing. 
 
Consider the ﬁlm's emotional tone, use of historical context, and its treatment of national 
trauma.  
How does it contribute to Japan's postwar narrative of victimhood? Do you ﬁnd any social 
criticism? 
 
The ﬁlm contributes to postwar Japan's victimhood narrative by showing wartime events 
through the perspective of a non-combatant. 
While one could argue that soldi

In [ ]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline, LightPipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    YakeKeywordExtraction
)
import pyspark.sql.functions as F
# Step 1: Transforms raw texts to `document` annotation
document = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("document")
# Step 2: Sentence Detection
sentenceDetector = SentenceDetector() \
            .setInputCols("document") \
            .setOutputCol("sentence")
# Step 3: Tokenization
token = Tokenizer() \
            .setInputCols("sentence") \
            .setOutputCol("token") \
            .setContextChars(["(", ")", "?", "!", ".", ","])
# Step 4: Keyword Extraction
keywords = YakeKeywordExtraction() \
            .setInputCols("token") \
            .setOutputCol("keywords") \

# Define the pipeline
yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])
# Create an empty dataframe
empty_df = spark.createDataFrame([['']]).toDF("text")
# Fit the dataframe to get the
yake_Model = yake_pipeline.fit(empty_df)

In [ ]:

light_model = LightPipeline(yake_Model)
text = data
light_result = light_model.fullAnnotate(text)[0]

In [ ]:
keys_df = pd.DataFrame([(k.result, k.begin, k.end, k.metadata['score'],  k.metadata['sentence']) for k in light_result['keywords']],
                       columns = ['keywords','begin','end','score','sentence'])
keys_df['score'] = keys_df['score'].astype(float)
# ordered by relevance
keys_df.sort_values(['sentence','score']).head(100)

,keywords,begin,end,score,sentence
0,fireﬂies notes,13,26,0.292945,0
2,human cost,58,67,0.395698,0
26,cost of war,64,74,0.990088,0
27,lens of children,102,117,1.513935,0
1,ﬁlm portray,42,52,1.807949,0
25,portray the human,46,62,2.658581,0
3,human cost,127,136,0.395698,1
28,cost of war,133,143,0.990088,1
4,portrayed brutally,148,165,2.229786,1
29,war is portrayed,141,156,2.857569,1


In [ ]:
!pip install groq

In [ ]:
import os

from groq import Groq

GROQ_API_KEY = "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" # Custom API Key 'adeasis'

input = f"With these notes {data}, make some key study questions/answers that we will port to flashcards, and store them. Label questions with 'Q:', and answers with 'A:'." # user prompt


# API key passes to GROQ Constructro Function
client = Groq(
    api_key=GROQ_API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": input,
        }
    ],
    model="llama3-8b-8192",
)
output = chat_completion.choices[0].message.content
complete_output = output.replace('*', '')
#output = output.replace('\n', '')
print(complete_output)


Here are the study questions and answers based on the notes:

Q: How does the film portray the human cost of war, particularly through the lens of children?
A: The film portrays the human cost of war brutally and realistically, showing the intense shock and trauma of losing family at a young age.

Q: What do we see through the lens of children in the film?
A: We see the unnecessary death toll of war, as well as the possibility of losing family without warning or preparation, which is life-changing for young children.

Q: How does the film contribute to Japan's postwar narrative of victimhood?
A: The film contributes to Japan's postwar victimhood narrative by showing wartime events through the perspective of a non-combatant, emphasizing the suffering of civilians rather than soldiers.

Q: Is there social criticism in the film?
A: Yes, the film contains social criticism, particularly in the character of Seita's aunt, who is an orphan and is treated unfairly despite the hardships she face

In [ ]:
# Helper functions

# Function to split strings based on entered string and substring
def splitfunc(str, q):
    return str.split(q, 1)

# Function to remove newlines from strings
def nlRemover(list):
  newList = []
  for item in list:
    newItem = item.replace('\n', '')
    newList.append(newItem)
  return newList

# Function to return question string, answer string, and remaining string after splitting
def qnasplit(str):
  Q = 'Q: '
  A = 'A: '
  # Isolating question
  split = splitfunc(str, Q)
  # Isolating answer
  splita = splitfunc(split[1], A)
  # Isolating next question
  splitb = splitfunc(splita[1], Q)
  question = splita[0]
  answer = splitb[0]
  if len(splitb) == 2:
    remaining = splitb[1]
  else:
    remaining = ''
  return question, answer, remaining

In [ ]:
# Function building a question/answer dictionary
def qna(str):
  notecard = {}
  Q = 'Q: '
  A = 'A: '
  nl = '\n'
  # Initial call of qnasplit
  q, a, r = qnasplit(str)
  notecard[q] = a
  # Was missing second question, brute force fix
  r2 = r
  second = splitfunc(r2, A)
  q = second[0]
  secondA = splitfunc(second[1], Q)
  a = secondA[0]
  notecard[q] = a
  # Calling qnasplit recursively
  while 'Q: ' in r and 'A: ' in r:
    q, a, r = qnasplit(r)
    notecard[q] = a
  # Exiting recursion when there is a remaining answer label, but no question label
  if 'A: ' in r:
    # Manually isolating final question and answer
    last = splitfunc(r, A)
    q = last[0]
    lastA = splitfunc(last[1], nl)
    a = lastA[0]
    notecard[q] = a
  return notecard

question, answer, remaining = qnasplit(complete_output)
q2, a2, r2 = qnasplit(remaining)


In [ ]:
notecard = qna(complete_output)
nckeys = notecard.keys()
questions = nlRemover(nckeys)
ncvalues = notecard.values()
answers = nlRemover(ncvalues)
lastAnswer = splitfunc(answers[-1], '.')
answers[-1] = lastAnswer[0] + '.'

#print(answers[-1])
print(questions)
print(answers)

['How does the film portray the human cost of war, particularly through the lens of children?', 'What do we see through the lens of children in the film?', "How does the film contribute to Japan's postwar narrative of victimhood?", "What is the significance of Seita's aunt's character in the film?"]
['The film portrays the human cost of war brutally and realistically, showing the intense shock and trauma of losing family at a young age.', 'We see the unnecessary death toll of war, as well as the possibility of losing family without warning or preparation, which is life-changing for young children.', "The film contributes to Japan's postwar victimhood narrative by showing wartime events through the perspective of a non-combatant, emphasizing the suffering of civilians rather than soldiers.", "Seita's aunt's character highlights the struggles and hardships faced by civilians during the war, and criticizes the treatment of orphans and those in need."]
